In [8]:
import numpy as np

In [9]:
from NeuronalNetwork import NeuralLayer, NeuralNetwork
from LossFunc import binary_crossentropy_derivative, binary_crossentropy
from ActivFunc import relu, relu_derivative, softmax, softmax_derivative
from Optimizer import SGD
import Datasets

In [10]:
def build_neural_network(X_train, y_train):
    # Arquitectura de la red neuronal
    input_size = X_train.shape[1]
    output_size = y_train.shape[1]
    hidden_size1 = 100
    hidden_size2 = 100

    # Creación de capas
    input_layer = NeuralLayer(input_size, hidden_size1, relu, relu_derivative)
    hidden_layer1 = NeuralLayer(hidden_size1, hidden_size2, relu, relu_derivative)
    hidden_layer2 = NeuralLayer(hidden_size2, output_size, softmax, softmax_derivative, use_bias=False)

    # Creación de la red neuronal
    neural_network = NeuralNetwork([input_layer, hidden_layer1, hidden_layer2])
    return neural_network

In [11]:
def train_neural_network(neural_network, X_train, y_train, X_test, y_test, optimizer, epochs=100, batch_size=20, momentum=0.9):
    # Entrenamiento
    for epoch in range(epochs):
        indices = np.arange(X_train.shape[0])
        np.random.shuffle(indices) # reorganiza de manera aleatoria el indice

        for i in range(0, X_train.shape[0], batch_size):
            batch_indices = indices[i:i+batch_size]
            X_batch = X_train[batch_indices]
            y_batch = y_train[batch_indices]

            # Forward pass
            predicted_output = neural_network.forward(X_batch)

            # Cálculo de la pérdida
            loss = np.mean(binary_crossentropy(y_batch, predicted_output))

            # Backpropagation
            error = binary_crossentropy_derivative(y_batch, predicted_output)
            neural_network.backward(error, optimizer, momentum)

        # Evaluación en el conjunto de prueba
        test_output = neural_network.forward(X_test)
        test_loss = np.mean(binary_crossentropy(y_test, test_output))

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss:.4f}, Test Loss: {test_loss:.4f}")


In [12]:
def evaluate_final_accuracy(neural_network, X_test, y_test):
    # Evaluación final en el conjunto de prueba
    final_output = neural_network.forward(X_test)
    final_accuracy = np.mean(np.argmax(final_output, axis=1) == np.argmax(y_test, axis=1))
    print(f"Final Accuracy on Test Set: {final_accuracy * 100:.2f}%")


In [13]:
def main():
    # Carga y preprocesamiento de datos
    X_train, y_train, X_test, y_test = Datasets.load_and_preprocess_data_mnist("") # añadir direccion de la carpeta con datos MNIST

    #X_train, y_train, X_test, y_test = Datasets.load_and_preprocess_data_iris()

    # Construcción de la red neuronal
    neural_network = build_neural_network(X_train, y_train)

    # Entrenamiento de la red neuronal
    optimizer = SGD(learning_rate=0.01)
    train_neural_network(neural_network, X_train, y_train, X_test, y_test, optimizer, epochs=200, batch_size=150, momentum=0.9)

    # Evaluación final en el conjunto de prueba
    evaluate_final_accuracy(neural_network, X_test, y_test)

In [14]:
if __name__ == "__main__":
    main()

Epoch 1/200, Loss: 0.0718, Test Loss: 0.0451
Epoch 2/200, Loss: 0.0279, Test Loss: 0.0358
Epoch 3/200, Loss: 0.0216, Test Loss: 0.0296
Epoch 4/200, Loss: 0.0240, Test Loss: 0.0251
Epoch 5/200, Loss: 0.0295, Test Loss: 0.0237
Epoch 6/200, Loss: 0.0240, Test Loss: 0.0211
Epoch 7/200, Loss: 0.0090, Test Loss: 0.0200
Epoch 8/200, Loss: 0.0158, Test Loss: 0.0182
Epoch 9/200, Loss: 0.0218, Test Loss: 0.0168
Epoch 10/200, Loss: 0.0057, Test Loss: 0.0151
Epoch 11/200, Loss: 0.0144, Test Loss: 0.0155
Epoch 12/200, Loss: 0.0065, Test Loss: 0.0152
Epoch 13/200, Loss: 0.0136, Test Loss: 0.0149
Epoch 14/200, Loss: 0.0096, Test Loss: 0.0139
Epoch 15/200, Loss: 0.0179, Test Loss: 0.0147
Epoch 16/200, Loss: 0.0085, Test Loss: 0.0136
Epoch 17/200, Loss: 0.0061, Test Loss: 0.0137
Epoch 18/200, Loss: 0.0162, Test Loss: 0.0141
Epoch 19/200, Loss: 0.0048, Test Loss: 0.0141
Epoch 20/200, Loss: 0.0058, Test Loss: 0.0131
Epoch 21/200, Loss: 0.0035, Test Loss: 0.0137
Epoch 22/200, Loss: 0.0040, Test Loss: 0.01